In [7]:
# Imports
import cv2
import glob
import time
import matplotlib.image as mpimg
import numpy as np

from skimage.feature import hog
from skimage import color, exposure
from sklearn.calibration import CalibratedClassifierCV
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

import features as F


In [2]:
# Utility functions for feature engineering

    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


In [3]:
# Load Data and Feature engineering.

def LoadData(imagePath):
    images = glob.glob(imagePath)
    data = []

    for image in images:
        data.append(image)
    return data


carPath = "./data/vehicles/*/*.png"
notCarPath = "./data/non-vehicles/*/*.png"
cars = LoadData(carPath)
notCars = LoadData(notCarPath)

# Prepare Features.
print("Number of vehicles images: ", len(cars))
print("Number of non-vehicles images: ", len(notCars))

f = F.Features()

car_features = f.extract_features(cars)
notcar_features = f.extract_features(notCars)



Number of vehicles images:  8792
Number of non-vehicles images:  8968


In [9]:
# Train and Save Classifier

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC(C=1.0)
clf = CalibratedClassifierCV(svc)
# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))
print('Train Accuracy of SVC = ', round(clf.score(X_train, y_train), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', clf.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
print('with probability: ', clf.predict_proba(X_test[0:n_predict]))

t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
# Save model & scaler
print("Saving models...")
joblib.dump(svc, './models/classifier.pkl')
joblib.dump(clf, './models/calibrated.pkl')
joblib.dump(X_scaler, './models/scaler.pkl')
print("Done!")

Feature vector length: 8460
69.42 Seconds to train SVC...
Test Accuracy of SVC =  0.9837
Train Accuracy of SVC =  1.0
My SVC predicts:  [ 0.  0.  0.  0.  1.  1.  1.  1.  0.  1.]
For these 10 labels:  [ 0.  0.  0.  0.  1.  1.  1.  1.  0.  1.]
with probability:  [[  9.97381891e-01   2.61810925e-03]
 [  9.99944992e-01   5.50078826e-05]
 [  9.99770059e-01   2.29940975e-04]
 [  9.97633018e-01   2.36698192e-03]
 [  6.86829604e-04   9.99313170e-01]
 [  2.37877632e-01   7.62122368e-01]
 [  3.65420073e-04   9.99634580e-01]
 [  1.30393157e-06   9.99998696e-01]
 [  9.94556179e-01   5.44382100e-03]
 [  5.36131702e-04   9.99463868e-01]]
0.00894 Seconds to predict 10 labels with SVC
Saving models...
Done!


In [10]:
# Load Models

# Load the classifier model
clf = joblib.load('./models/classifier.pkl')
cal = joblib.load('./models/calibrated.pkl')

# Load the standard scalar model
X_scaler = joblib.load('./models/scaler.pkl')

In [ ]:
def process_image(img):

    return img


In [ ]:
clip = VideoFileClip("./project_video.mp4")
output_video = "./project_video_marked.mp4"
output_clip = clip.fl_image(process_image)
output_clip.write_videofile(output_video, audio=False)
